## Dengai Model

In this model I will first build a xgboost model and then pass these predictions along with the features to a neural network, the idea being that that the initial model will aid the neural network in making a more precise estimate.


In [5]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel, VarianceThreshold



#### Preprocess Data

This function preprocesses the data, fills na values and separates the data from each city, all features are saved.
It is able to distinguish when you are loading just the test data and the train data.

In [6]:
# make function to preprocess data
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path)
    
    # fill missing values
    # try instead using mean and median
    df.fillna(method='bfill', inplace=True)
    
    #df = df[np.notnull(df)]
    #df = df.dropna()

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path)
        #df = df.join(labels)
    
    # separate san juan and iquitos
    sj_features = df[df.city == 'sj']
    iq_features = df[df.city == 'iq']
    
    #dropping date and city as city already divided
    iq_features = iq_features.drop(iq_features.columns[[0,3]], axis=1)
    sj_features = sj_features.drop(sj_features.columns[[0,3]], axis=1)


#sj_labels.head()
    if labels_path:
        sj_labels = labels[labels.city == 'sj']
        iq_labels = labels[labels.city == 'iq']   
        #removing city, year, weekofyear from labels tables
        sj_labels = sj_labels.total_cases
        iq_labels = iq_labels.total_cases
        return sj_features, iq_features, sj_labels, iq_labels
    return sj_features, iq_features

In [7]:
sj_features, iq_features, sj_labels, iq_labels = preprocess_data(
                                                                'data/dengue_features_train.csv',
                                                                labels_path="data/dengue_labels_train.csv")

In [8]:
#load final test data
sj_test_final, iq_test_final = preprocess_data("data/dengue_features_test.csv")

Since data is already divided by city I remove that column as well as the date column as other columns represent it, so it is kinda redundant, as well as python doesn't like its string formatting

## Features and their descriptions
copied from the example website

#### City and date indicators
city – City abbreviations: sj for San Juan and iq for Iquitos
week_start_date – Date given in yyyy-mm-dd format

#### NOAA's GHCN daily climate data weather station measurements
station_max_temp_c – Maximum temperature
station_min_temp_c – Minimum temperature
station_avg_temp_c – Average temperature
station_precip_mm – Total precipitation
station_diur_temp_rng_c – Diurnal temperature range

#### PERSIANN satellite precipitation measurements (0.25x0.25 degree scale)
precipitation_amt_mm – Total precipitation

#### NOAA's NCEP Climate Forecast System Reanalysis measurements (0.5x0.5 degree scale)
<p>reanalysis_sat_precip_amt_mm – Total precipitation
reanalysis_dew_point_temp_k – Mean dew point temperature
reanalysis_air_temp_k – Mean air temperature
reanalysis_relative_humidity_percent – Mean relative humidity
reanalysis_specific_humidity_g_per_kg – Mean specific humidity
reanalysis_precip_amt_kg_per_m2 – Total precipitation
reanalysis_max_air_temp_k – Maximum air temperature
reanalysis_min_air_temp_k – Minimum air temperature
reanalysis_avg_temp_k – Average air temperature
reanalysis_tdtr_k – Diurnal temperature range
</p>

#### Satellite vegetation - Normalized difference vegetation index (NDVI) - NOAA's CDR Normalized Difference Vegetation Index (0.5x0.5 degree scale) measurements
ndvi_se – Pixel southeast of city centroid
ndvi_sw – Pixel southwest of city centroid
ndvi_ne – Pixel northeast of city centroid
ndvi_nw – Pixel northwest of city centroid

In [9]:
sj_features.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [166]:
#add lagging variables to most significant variables
Cols = sj_features.columns.values.tolist()
clf = GradientBoostingRegressor(random_state = 8001)

selector = clf.fit(sj_features, sj_labels)
importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
train = fs.transform(sj_features)
print(train.shape)

(936, 5)


In [10]:
#randomly separating data
# splitting data into training set and test set

sj_train, sj_test, sj_train_target, sj_test_target = train_test_split(sj_features, sj_labels, test_size=0.2, random_state=41)

iq_train, iq_test, iq_train_target, iq_test_target = train_test_split(iq_features, iq_labels, test_size=0.5, random_state=41)



-------------------------------------------------------------------------------------------------------------------
Using K-Folds
-----------------------------------------------------------------------------------------------------------

In [97]:
sj_predictions = np.zeros(sj_features.shape[0])
len(sj_predictions)

936

In [98]:
kf = KFold(sj_features.shape[0], n_folds=5)



In [99]:
for trainIndex, testIndex in kf:
    trainFold, testFold = sj_features.iloc[trainIndex], sj_features.iloc[testIndex]
    trainFoldTarget, testFoldTarget = sj_labels.iloc[trainIndex], sj_labels.iloc[testIndex]
    
    xgbr = xgb.XGBRegressor(n_estimators = 750, # number of boosted trees
                                learning_rate = 0.0015, # step size shrinkage used in update to prevent overfitting
                                max_depth = 7,
                                subsample = 0.8, # subsample ratio of the training set (Stochastic gradient boosting)
                                colsample_bytree = 0.701)
    
    xgbr.fit(trainFold, trainFoldTarget)
    xgbpred =xgbr.predict(testFold)
    #testPred.append(xgbr.predict(test))
    sj_predictions[testIndex] = xgbpred
    
    # Print the AUC
    print(metrics.mean_absolute_error(testFoldTarget, xgbpred))


24.8121840041
36.4674940058
31.1182927167
10.4341155259
16.055401899


In [100]:
def xboostRegressor(city_feat, city_labels):
    '''
    this function builds a xboost model given a city
    '''
    xgbr = xgb.XGBRegressor(n_estimators = 750, # number of boosted trees
                                learning_rate = 0.0015, # step size shrinkage used in update to prevent overfitting
                                max_depth = 7,
                                subsample = 0.8, # subsample ratio of the training set (Stochastic gradient boosting)
                                colsample_bytree = 0.701)
    
    predictions = np.zeros(city_feat.shape[0])
    
    kf = KFold(city_feat.shape[0], n_folds=5)
    
    for trainIndex, testIndex in kf:
        trainFold, testFold = city_feat.iloc[trainIndex], city_feat.iloc[testIndex]
        trainFoldTarget, testFoldTarget = city_labels.iloc[trainIndex], city_labels.iloc[testIndex]
        xgbr.fit(trainFold, trainFoldTarget)
        xgbpred =xgbr.predict(testFold)
        #testPred.append(xgbr.predict(test))
        sj_predictions[testIndex] = xgbpred
    return xgbr
    
    # Print the AUC
    #print(metrics.mean_absolute_error(testFoldTarget, xgbpred))

In [101]:
print(metrics.mean_absolute_error(sj_labels, sj_predictions))
#obviously this number is going to be really small since we tested our data with the same data we used for 
#training

23.7786030645


#### This function uses training-testing split sets



In [11]:
def xboostRegressor(city_feat, city_labels):
    '''
    this function builds a xboost model given a city
    '''        
    xgbr = xgb.XGBRegressor(n_estimators = 750, # number of boosted trees
                                learning_rate = 0.003057, # step size shrinkage used in update to prevent overfitting
                                max_depth = 10,
                                subsample = 0.75, # subsample ratio of the training set (Stochastic gradient boosting)
                                colsample_bytree = 0.75,
                               gamma = .025)
    xgbr.fit(city_feat,city_labels)
    return xgbr
    
    # Print the AUC
    #print(metrics.mean_absolute_error(testFoldTarget, xgbpred))

In [12]:
sj_model = xboostRegressor(sj_train, sj_train_target)
iq_model = xboostRegressor(iq_train, iq_train_target)

In [16]:
sj_pred = sj_model.predict(sj_test)
score = metrics.mean_absolute_error(sj_test_target,sj_pred)
print(score)

13.2535311085


In [17]:
iq_pred = iq_model.predict(iq_test)
score = metrics.mean_absolute_error(iq_test_target,iq_pred)
print(score)

5.9157173504


In [41]:
#adding predictions to featurers
sj_pred = sj_model.predict(sj_features)
iq_pred = iq_model.predict(iq_features)

#convert from float to int
sj_pred = [int(i) for i in sj_pred]
iq_pred = [int(i) for i in iq_pred]


sj_features['xgb_pred'] = list(sj_pred)
iq_features['xgb_pred'] = list(iq_pred)

score = metrics.mean_absolute_error(sj_labels,sj_pred)


7.51282051282


In [39]:
sj_features.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,xgb_pred
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,9
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,5


In [52]:
#normalize features table
from sklearn import preprocessing

sj_features.ndvi_ne = preprocessing.scale(sj_features.ndvi_ne)
sj_features.ndvi_nw = preprocessing.scale(sj_features.ndvi_nw)
sj_features.ndvi_ne = preprocessing.scale(sj_features.ndvi_ne)


for column in sj_features:
    notnorm = ['year','weekofyear','xgb_pred']
    if column not in notnorm:
        sj_features[column] = preprocessing.scale(sj_features[column])


sj_features.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,xgb_pred
0,1990,18,0.585062,0.410758,0.360550,0.205706,-0.514378,-1.281232,-1.252891,-1.713274,...,-1.535143,-0.514378,-1.623318,0.230403,-1.097037,0.175971,-1.275173,-1.723747,-0.369395,9
1,1990,19,1.016554,0.829255,-0.274076,-0.190959,-0.282134,-0.764552,-0.678411,-0.733529,...,-0.353824,-0.282134,-0.751434,-0.285729,-0.199382,-0.455032,0.060752,-0.261587,-0.622480,5
2,1990,20,-0.239150,1.164397,-0.364670,0.084296,-0.020413,-0.303353,-0.320827,0.211615,...,1.028598,-0.020413,0.194632,-0.429099,-0.199382,-0.318599,0.351170,0.137184,0.499299,4
3,1990,21,0.640101,1.949147,0.871286,1.250091,-0.448725,-0.136906,-0.033588,0.132397,...,0.522258,-0.448725,0.081984,-0.171033,0.335177,0.022484,0.990091,0.469493,-0.779802,3
4,1990,22,1.256474,2.135629,1.286618,1.455392,-0.623801,0.293084,0.323996,0.458372,...,0.558515,-0.623801,0.426341,1.004600,1.374036,3.126335,1.977514,0.868264,-0.718241,5


In [53]:
sj_labels.head()

0    4
1    5
2    4
3    3
4    6
Name: total_cases, dtype: int64

In [157]:

final_sj_predictions = sj_model.predict(sj_test_final)
final_iq_predictions = iq_model.predict(iq_test_final)

In [158]:
submission = pd.read_csv("data/dengue_labels_test.csv",
                         index_col=[0, 1, 2])


submission.total_cases = np.concatenate([final_sj_predictions.astype(np.int64), final_iq_predictions.astype(np.int64)])
submission.to_csv("submission/submission_xgboost.csv")


In [ ]:
#submission